In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from tqdm import tqdm

In [44]:
# Function to create scrollable table within a small window
def create_scrollable_table(df, table_id, title):
    html = f'<h3>{title}</h3>'
    html += f'<div id="{table_id}" style="height:200px; overflow:auto;">'
    html += df.to_html()
    html += '</div>'
    return html

In [110]:
training = pd.read_csv('C:/Users/hangu/Creative Cloud Files/0. Data Science study/Datasets/Home Credit default risk/application_train.csv')
testing = pd.read_csv('C:/Users/hangu/Creative Cloud Files/0. Data Science study/Datasets/Home Credit default risk/application_test.csv')
bureau = pd.read_csv('C:/Users/hangu/Creative Cloud Files/0. Data Science study/Datasets/Home Credit default risk/bureau.csv')                      # previous credits in Credit Bureau
bureau_balance = pd.read_csv('C:/Users/hangu/Creative Cloud Files/0. Data Science study/Datasets/Home Credit default risk/bureau_balance.csv')      # monthly balance
                                                                                                                                                    # 'STATUS' meaning?
POS_CASH = pd.read_csv('C:/Users/hangu/Creative Cloud Files/0. Data Science study/Datasets/Home Credit default risk/POS_CASH_balance.csv')          # previous POS (point of sales) and cash loans 
credit_card = pd.read_csv('C:/Users/hangu/Creative Cloud Files/0. Data Science study/Datasets/Home Credit default risk/credit_card_balance.csv')    # previous credit cards
prev_apl = pd.read_csv('C:/Users/hangu/Creative Cloud Files/0. Data Science study/Datasets/Home Credit default risk/previous_application.csv')      # previous applications
payment = pd.read_csv('C:/Users/hangu/Creative Cloud Files/0. Data Science study/Datasets/Home Credit default risk/installments_payments.csv')      # Repayment history for the previously disbursed credits


In [117]:
print(bureau.columns, "\n")
pd.DataFrame(bureau['SK_ID_CURR'].value_counts()).transpose()

# SK_BUREAU_ID: unique in this "bureau" df
# DAYS_CREDIT: How many days before current application did client apply for Credit Bureau credit
# CREDIT_DAY_OVERDUE: Number of days past due on CB credit at the time of application for related loan in our sample
# DAYS_CREDIT_ENDDATE: Remaining duration of CB credit (in days) at the time of application in Home Credit
# DAYS_CREDIT_UPDATE: How many days before loan application did last information about the Credit Bureau credit come

Index(['SK_ID_CURR', 'SK_ID_BUREAU', 'CREDIT_ACTIVE', 'CREDIT_CURRENCY',
       'DAYS_CREDIT', 'CREDIT_DAY_OVERDUE', 'DAYS_CREDIT_ENDDATE',
       'DAYS_ENDDATE_FACT', 'AMT_CREDIT_MAX_OVERDUE', 'CNT_CREDIT_PROLONG',
       'AMT_CREDIT_SUM', 'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM_LIMIT',
       'AMT_CREDIT_SUM_OVERDUE', 'CREDIT_TYPE', 'DAYS_CREDIT_UPDATE',
       'AMT_ANNUITY'],
      dtype='object') 



SK_ID_CURR,120860,169704,318065,251643,425396,295809,177014,280155,252366,218175,...,304748,430469,160781,334642,127041,234522,180076,171655,258946,399573
count,116,81,78,61,60,59,56,55,55,55,...,1,1,1,1,1,1,1,1,1,1


In [6]:
print(bureau_balance.columns, "\n")
pd.DataFrame(bureau_balance['SK_ID_BUREAU'].value_counts()).transpose()     # Number of Months in balance = count - 1

# MONTHS_BALANCE: Month of balance relative to application date (-1 means the freshest balance date)
# STATUS: Status of Credit Bureau loan during the month (C means closed, X means status unknown, 0 means no DPD, 1 means maximal did during month between 1-30, 2 means DPD 31-60,… 5 means DPD 120+ or sold or written off ] )

Index(['SK_ID_BUREAU', 'MONTHS_BALANCE', 'STATUS'], dtype='object') 



SK_ID_BUREAU,5907926,6786422,6028907,5229867,6363619,5002065,5817528,5817489,5817435,5002064,...,5253267,5708683,5709203,5708684,5242749,5052730,5207756,6543954,6364503,5005798
count,97,97,97,97,97,97,97,97,97,97,...,1,1,1,1,1,1,1,1,1,1


In [119]:
print(POS_CASH.columns, "\n")

POS_CASH.sort_values(by='MONTHS_BALANCE')

Index(['SK_ID_PREV', 'SK_ID_CURR', 'MONTHS_BALANCE', 'CNT_INSTALMENT',
       'CNT_INSTALMENT_FUTURE', 'NAME_CONTRACT_STATUS', 'SK_DPD',
       'SK_DPD_DEF'],
      dtype='object') 



,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
767082,1775141,244572,-96,4.0,2.0,Active,0,0
3573052,1595167,454443,-96,10.0,3.0,Active,17,0
5231758,2588917,294677,-96,4.0,2.0,Active,0,0
2251801,1810403,145248,-96,10.0,0.0,Active,0,0
727339,1925384,330163,-96,12.0,6.0,Active,0,0
...,...,...,...,...,...,...,...,...
3887454,2385634,121591,-1,48.0,27.0,Active,0,0
2388448,1335383,186424,-1,48.0,38.0,Active,0,0
7348487,2318117,271325,-1,10.0,8.0,Active,0,0
871138,1900506,208057,-1,12.0,10.0,Active,0,0


In [101]:
sample = POS_CASH.groupby(['SK_ID_CURR', 'SK_ID_PREV'])['MONTHS_BALANCE'].min()

df_sample = pd.DataFrame(sample)
df_sample = df_sample.reset_index()
df_sample

,SK_ID_CURR,SK_ID_PREV,MONTHS_BALANCE
0,100001,1369693,-57
1,100001,1851984,-96
2,100002,1038818,-19
3,100003,1810518,-25
4,100003,2396755,-77
...,...,...,...
936320,456255,1359084,-15
936321,456255,1743609,-33
936322,456255,2073384,-21
936323,456255,2631384,-26


In [106]:
a = POS_CASH[POS_CASH['SK_ID_CURR']==100001]
a[a['SK_ID_PREV']==1369693].sort_values(by='MONTHS_BALANCE')

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
7167007,1369693,100001,-57,4.0,4.0,Active,0,0
8789081,1369693,100001,-56,4.0,3.0,Active,0,0
7823681,1369693,100001,-55,4.0,2.0,Active,0,0
4704415,1369693,100001,-54,4.0,1.0,Active,0,0
2197888,1369693,100001,-53,4.0,0.0,Completed,0,0


In [109]:
df_sample[df_sample['SK_ID_CURR']==265042]

,SK_ID_CURR,SK_ID_PREV,MONTHS_BALANCE
433077,265042,1130567,-20
433078,265042,1162724,-70
433079,265042,1211034,-49
433080,265042,1295332,-90
433081,265042,1295333,-79
433082,265042,1330669,-54
433083,265042,1364905,-36
433084,265042,1510354,-96
433085,265042,1570060,-54
433086,265042,1735921,-85


In [60]:
col_list = ['SK_ID_PREV', 'SK_ID_CURR', 'MONTHS_BALANCE']
credit_card = credit_card[col_list].sort_values(by='MONTHS_BALANCE')
credit_card

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE
24268,2191008,186722,-96
841327,1462198,437589,-96
2725728,2181176,123766,-96
2725726,1365551,162637,-96
841332,1619825,271412,-96
...,...,...,...
395926,2116047,243443,-1
395931,1396266,163805,-1
395938,1468878,269362,-1
395901,1367649,167456,-1


In [97]:
sample = credit_card.groupby(['SK_ID_CURR', 'SK_ID_PREV'])['MONTHS_BALANCE'].min()

df_sample = pd.DataFrame(sample)
df_sample = df_sample.reset_index()
df_sample

,SK_ID_CURR,SK_ID_PREV,MONTHS_BALANCE
0,100006,1489396,-6
1,100011,1843384,-75
2,100013,2038692,-96
3,100021,2594025,-18
4,100023,1499902,-11
...,...,...,...
104302,456244,2181926,-41
104303,456246,1079732,-9
104304,456247,1595171,-96
104305,456248,2743495,-24


In [98]:
df_sample[df_sample['SK_ID_CURR']==186401]

,SK_ID_CURR,SK_ID_PREV,MONTHS_BALANCE
25108,186401,1440617,-96
25109,186401,2243411,-96


In [115]:
prev_apl

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,...,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,...,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1670209,2300464,352015,Consumer loans,14704.290,267295.5,311400.0,0.0,267295.5,WEDNESDAY,12,...,Furniture,30.0,low_normal,POS industry with interest,365243.0,-508.0,362.0,-358.0,-351.0,0.0
1670210,2357031,334635,Consumer loans,6622.020,87750.0,64291.5,29250.0,87750.0,TUESDAY,15,...,Furniture,12.0,middle,POS industry with interest,365243.0,-1604.0,-1274.0,-1304.0,-1297.0,0.0
1670211,2659632,249544,Consumer loans,11520.855,105237.0,102523.5,10525.5,105237.0,MONDAY,12,...,Consumer electronics,10.0,low_normal,POS household with interest,365243.0,-1457.0,-1187.0,-1187.0,-1181.0,0.0
1670212,2785582,400317,Cash loans,18821.520,180000.0,191880.0,NaN,180000.0,WEDNESDAY,9,...,XNA,12.0,low_normal,Cash X-Sell: low,365243.0,-1155.0,-825.0,-825.0,-817.0,1.0


In [116]:
payment

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585
...,...,...,...,...,...,...,...,...
13605396,2186857,428057,0.0,66,-1624.0,NaN,67.500,NaN
13605397,1310347,414406,0.0,47,-1539.0,NaN,67.500,NaN
13605398,1308766,402199,0.0,43,-7.0,NaN,43737.435,NaN
13605399,1062206,409297,0.0,43,-1986.0,NaN,67.500,NaN


In [8]:
training

# TARGET: (1) client with payment difficulties, (0) all other cases

# DAYS_BIRTH: Client's age in days at the time of application
# DAYS_EMPLOYED: How many days before the application the person started current employment

# REGION_RATING_CLIENT: Our rating of the region where client lives (1,2,3)
# REGION_RATING_CLIENT_W_CITY: Our rating of the region where client lives with taking city into account (1,2,3)

# WEEKDAY_APPR_PROCESS_START: On which day of the week did the client apply for the loan

# AMT_REQ_CREDIT_BUREAU_YEAR: Number of enquiries to Credit Bureau about the client one year before application (excluding last 3 months before application)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307507,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307508,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,0,0,0,0,1.0,0.0,0.0,1.0,0.0,1.0
307509,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
testing

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48739,456221,Cash loans,F,N,Y,0,121500.0,412560.0,17473.5,270000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
48740,456222,Cash loans,F,N,N,2,157500.0,622413.0,31909.5,495000.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
48741,456223,Cash loans,F,Y,Y,1,202500.0,315000.0,33205.5,315000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,3.0,1.0
48742,456224,Cash loans,M,N,N,0,225000.0,450000.0,25128.0,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0
